# Agenda

1. What is a dictionary?
2. Hash functions
3. Creating a `HashTable` class
4. Retrievals from it
5. Assignments to it
6. Improving the search
7. Magic methods for making it more dict-like
8. Iteration over our dict
9. Modern Python dicts vs. this implementation

# What is a dict?

We in the Python world love dicts. But they're in other languages, too:

- Hashes
- Hash tables
- Hash maps
- Maps
- Key-value stores
- Name-value stores

the idea is that we have pairs of information that we're storing:

- We have the "key,"  we get to determine what it is
    - Keys are unique within a dict
    - They have to be (sort of) immutable
- We have values, which can be anything


In [1]:
d = {'a':10, 'b':20, 'c':30}
d

{'a': 10, 'b': 20, 'c': 30}

In [2]:
len(d)

3

In [3]:
type(d)

dict

In [4]:
d['a'] = 999
d

{'a': 999, 'b': 20, 'c': 30}

In [5]:
d['x']

KeyError: 'x'

In [6]:
# search in a dict for a key
'x' in d

False

In [7]:
'a' in d

True

In [9]:
# how do we remove a key-value pair from our dict?
# dict.pop

d.pop('a')  # removes the key-value pair, and returns the value we removed

KeyError: 'a'

# Rules associated with them

- Keys will be unique. There is no way for a key to repeat itself in a dict
- The keys must be *hashable* , which is sorta kinda the same as immutable.
- Values can be anything at all.

In [10]:
name = 'Reuven'

In [11]:
globals()['name']

'Reuven'

# Hashable and hash functions

If I want to search for an element in a list, how long will it take? It depends.

Dicts are far, far faster than this. The reason is a hash function. The location used by Python to store the key-value pair is based on the key. We run a function, `hash`, on the key, and that gives us a memory location.

If I say

    d['a'] = 100

Python computes `hash('a')`, and then stores ('a', 100) in that location.

In [12]:
hash('a')

-8342762356364650461

In [13]:
hash('b')

-2087163220244094577

In [14]:
hash('c')

3079803896062575534

In [15]:
'a' in d

False

In [16]:
# why doesn't Python let me do this?

mylist = [10, 20, 30]
d[mylist] = 'hello'

TypeError: unhashable type: 'list'

# Exercise: Write a hash function

1. Do not use the builtin function `hash`. Call it `myhash`.
2. It can be really simple!
3. It should work with inputs of type `int`, `float`, `string`, and `tuple`.
4. Given any of these inputs, the function should return an integer.
5. Any other type can raise a `TypeError`.
6. You can use `ord` (which gives the Unicode value for a character) as part of your calculation
7. You should get different values for `hash('abc')` as `hash('cba')`.

You can check the type of a value with `isinstance`, as in `isinstance(10, int)`

In [30]:
def myhash(key):
    if isinstance(key, str):
        total = 0
        for index, one_character in enumerate(key, 1):
            total += ord(one_character) * index
        return total
    elif isinstance(key, int):
        return key
    elif isinstance(key, float):
        return int(key)
    elif isinstance(key, tuple):
        total = 0
        for index, one_item in enumerate(key, 1):
            total += myhash(one_item) * index
        return total
    else:
        raise TypeError(f'Unhashable type: {type(key)}')

In [31]:
myhash('a')

97

In [32]:
myhash('b')

98

In [33]:
myhash('ab')

293

In [34]:
myhash('ba')

292

In [35]:
myhash(10)

10

In [36]:
myhash(123.456)

123

In [38]:
myhash((10, 20, 30))

140

In [39]:
myhash(('hello', 'goodbye'))

7579

# What's the connection between `myhash` and a dict?

- When we assign a key-value pair to our dict, we will use `myhash` to know at what place to put it
- Our dict will start with only 8 locations in which we can put key-value pairs. We'll take the output from `myhash` and use `% 8` on it in order to place the pair.

In [41]:
myhash('abcd') % 8   # remainder after integer division by 8

6

# Exercise: Create a `HashTable` class

1. When you create a new instance of `HashTable`, you'll pass a list of 2-element tuples. The first element will be the key, and the second will be the value.
2. Allocate a list of 8 `None` values in your instance.
3. Iterate over each element in your input list of tuples, hashing the key and putting the key-value pair in the right place.
4. This might end up with some data loss!

Example:

    ht = HashTable([('a',10), ('b',20), ('c',30)])
    print(ht.data)  # we should see a list of 8 elements, all None except 
           # for our 3 pairs

In [42]:
class HashTable:
    def __init__(self, list_of_pairs):
        self.data = [None] * 8

        for one_pair in list_of_pairs:
            key, value = one_pair
            key_index = myhash(key) % 8
            self.data[key_index] = (key, value)

ht = HashTable([('a', 10), ('b', 20), ('c', 30)])            
            

In [43]:
ht.data

[None, ('a', 10), ('b', 20), ('c', 30), None, None, None, None]